# 1. Establish communication with the card

## 1.1. Find the smartcard reader

In [135]:
from smartcard import ATR
from smartcard.System import readers
from smartcard.CardConnection import CardConnection

In [136]:
rarr=readers()
if len(rarr) == 0:
    print("No readers detected")
else:
    reader = rarr[0]
    print(reader)

Generic Smart Card Reader Interface


## 1.2. Power on the card and get `ATR`
`ATR` - Answer To Reset, bytes retured by the card when it is powered on.<br>
It helps to determine communication protocol and frequency we should choose.

In [137]:
# We know that j3h145-jcop3 card supports T1 protocol, so we use that ignoring ATR
protocol = CardConnection.T1_protocol
connection = reader.createConnection()
connection.connect(protocol)

In [138]:
atr=bytes(connection.getATR())
atr.hex()

'3bdc18ff8191fe1fc38073c821136605036351000250'

## 1.3. Select the applet by AID
JavaCard can have multiple applets installed. They are like very isolated "apps".<br>
They can't talk to each other and only one can be active at any time.

In [139]:
def select(appletID):
    """
    Selects an applet by AppletID (should be hex-encoded string).
    Throws an error with the error code if it failed selecting the applet.
    Useful link for error codes: https://www.eftlab.com/knowledge-base/complete-list-of-apdu-responses/
    """
    data = list(bytes.fromhex(appletID))
    # Select:
    # CLA = 0x00
    # INS = 0xA4
    # P1 = 0x04
    # P2 = 0x00
    # Data = the instance AID
    cmd = [0x00, # CLA
              0xA4, # INS
              0x04, # P1
              0x00, # P2
              len(data), # Lc (content length)
          ] + data + [0x00]
    print("Sent command:", bytes(cmd).hex().upper())
    data, *sw = connection.transmit(cmd,protocol)
    data = bytes(data)
    sw = bytes(sw)
    if sw == b"\x90\x00":
        return data
    else:
        raise RuntimeError("Card responded with code %s and data \"%s\"" % (sw.hex(), data.hex()))

In [140]:
# MemoryCardApplet has AID of `B00B5111CB01`
select("B00B5111CB01")

Sent command: 00A4040006B00B5111CB0100


b''

In [141]:
def request(APDU):
    """
    Sends APDU command to the card and returns data.
    APDU should be hex-encoded.
    Throws an error with the error code if it failed selecting the applet.
    Useful link for error codes: https://www.eftlab.com/knowledge-base/complete-list-of-apdu-responses/
    """
    cmd = list(bytes.fromhex(APDU))
    data, *sw = connection.transmit(cmd,protocol)
    data = bytes(data)
    sw = bytes(sw)
    if sw == b"\x90\x00":
        return data
    else:
        raise RuntimeError("Card responded with code %s and data \"%s\"" % (sw.hex(), data.hex()))

# 2. Establish secure communication
This part is required to establish a shared secret between the card and the host.<br>
Because we want encrypted and authenticated communication.<br>

For key agreement we use ECDH - get card's pubkey, multiply by our secret, use x-coord of the resulting point as shared secret. Maybe sha256 of it?

## 2.1. Get public key from the card

In [142]:
# embit library: https://github.com/diybitcoinhardware/embit
# works with python3 and micropython
from embit.ec import PrivateKey, PublicKey, secp256k1

In [143]:
# get pubkey - private key is generated when applet is installed
sec = request("B0A1000000")
card_pubkey = PublicKey.parse(sec)
card_pubkey

PublicKey(042418faf5f140c6d2707fb1cb21d1ab7cbd94224dbdfd6fdb7fbb0891cddefa89ae2ab4fee4e673075c34ef472945f78a1c3125a725ad6a6024319bea5042fc20)

## 2.2. Select a random secret on the host

In [144]:
import os
import hashlib
secret = os.urandom(32)
# ugly copy...
pub = PublicKey.parse(card_pubkey.sec())
secp256k1.ec_pubkey_tweak_mul(pub._point, secret)
shared_secret = pub.sec()[1:33]
shared_hash = hashlib.sha256(shared_secret).digest()
print("Shared secret:", shared_secret.hex())
print("Sha256 of the secret:", shared_hash.hex())

Shared secret: 82666dd66ae73849e7ae54a759c60f6a23d1e2b6fb02e4dca6a8c368c8b55c74
Sha256 of the secret: 46a426b8bfe6fa4ac199cf375e4a5289e174f80cc144678163bbf2786e75e9c2


In [145]:
host_prv = PrivateKey(secret)
host_pub = host_prv.get_public_key()
host_pub.compressed = False
data = bytes([65])+host_pub.sec()
data.hex()

'4104c7ec69b9e7ac7523d4dfaabbbfc74fd27de8e42ea28020164f4f20e37829b6af0afba4f9fe7438634d8ecc2f4e75a7be086ff113623e4c01ef5484f0493e3e41'

In [146]:
res = request("B0A30000"+data.hex())
if res == shared_hash:
    print("Yey! We have a shared secret now!")
else:
    print("Meh... Something didn't work? Card returned %s, we have %s" % (res.hex(), shared_hash.hex()))

Yey! We have a shared secret now!


In [147]:
# get random data
rand = request("B0A2000000")
rand.hex()

'ed15ba2389a21d40ebd3102d3ba03d7bc5bbfbc8a588ad0fc9d1efe6a28e10c7'

In [148]:
# static data for any other APDU
data = request("B000000000")
data

b'Memory cards are not safu so what?'

In [149]:
(bytes([len(data)])+data).hex()

'224d656d6f727920636172647320617265206e6f74207361667520736f20776861743f'

In [150]:
# static data for any other APDU
c = request("B0A40000"+(bytes([len(data)])+data).hex())
c

b'a\xa1[\x03 \x8d\xa8\xa8\x9e_\xe1\xe3\xb7qB\x90\x9e\xdc\x84>\xe6\x99\xac\xda\tY\x9c\xd4"\x13H\xc2=\xf84\xe7\xbc\xfc1k\x80l<\xa5\x00X\xa4{I\xcf$\xbd"xZ\xbc1\x01\xb3\xb0\x9fL\xb9\xb9\xecB)\x9c\xd5\x96\x02\x1c\xc3\x94\x0f\xfb\x11\'\x06ec};\xec0=\xe0\xa9\x81m^TI\x96\xc10'

In [134]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

def decrypt(c,secret):
    hmac = c[-32:]
    iv = c[:16]
    ct = c[16:-32]
    h = hashlib.sha256(shared_secret)
    h.update(iv)
    h.update(ct)
    h.update(shared_secret)
    expected_hmac = h.digest()
    if expected_hmac != hmac:
        raise RuntimeError("Failed")
    backend = default_backend()
    cipher = Cipher(algorithms.AES(secret), modes.CBC(iv), backend=backend)
    decryptor = cipher.decryptor()
    # remove \x80... padding
    plain = decryptor.update(ct)+decryptor.finalize()
    return b"".join(plain.split(b"\x80")[:-1])

decrypt(c, shared_secret)

b'Memory cards are not safu so what?'

In [125]:
len(c)/16

4.0

In [38]:
connection.disconnect()